<a href="https://colab.research.google.com/github/raaraya1/Personal-Proyects/blob/main/Cursos/Fastai/Notas_de_la_clase8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clase 8 - Deep Learning for Coders**

# **Respuestas al cuestionario**

**1. ¿Qué es el "aprendizaje auto-supervisado"?**

El aprendizaje autosupervisado tiene por objetivo agrupar los elementos (generar categorias), pero sin llegar a conocer el nombre de la categoria por la cual se esta agrupando. En este sentido, este modelo se alimenta de informacion sin etiquetas (sin definir el target) y busca la mejor manera de ordenar los datos, de forma que, por ejemplo, en el predictor de palabras, al tener como entrada una cadena de texto, el modelo sea capaz de entregar la una palabra cercana a este grupo de palabras.

**2. ¿Qué es un "modelo de lenguaje"?**

Un modelo de lenguaje, es aquel modelo que fuen entrenado para predecir cual es la palabra siguiente en una cadena de texto.

**3. ¿Por qué un modelo de lenguaje se considera auto-supervisado?**

Se considera auto-supervisado, en el sentido de que es el propio modelo el que se encarga de etiquetar o agrupar los datos. (de darle cierta logica al ordenamiento)

**4. ¿Para qué se utilizan habitualmente los modelos autocontrolados?**

Estos suelen ser utilizados como modelos preentrenados y asi, realizar una transferencia de conocimiento.

**5. ¿Por qué ajustamos los modelos de lenguaje?**

Esto lo hacemos, ya que el modelo preentrenado maneja un cierto vocabulario de palabras pertenecientes a su training set. Sin embargo, si utilizamos este modelo preentrenado con un nuevo dataset, facilmente podrian aparecer nuevas palabras. Asi, este no sabria como tratar estos datos nuevos.

**6. ¿Cuáles son los tres pasos para crear un clasificador de texto de última generación?**

1. Tokenize el nuevo set de datos.
2. Realizar un proceso de `fit_one_cycle` al modelo de lenguaje con el nuevo set de datos (ya numerizados), luego de este se obtiene el Encoder.
3. Armar el modelo del clasificador (misma estructura que el modelo del lenguaje, salvo la capa de salida), 'pegarle' los parametros guardados en el Enconder. Por ultimo, ir actualizando de a poco las ultimas capas del modelo mediante los metodos `fit_one_cycle` y `freeze_to` y al final descongelar todo, para continuar entrenando. 

**7. ¿Cómo nos ayudan las 50.000 reseñas de películas sin etiquetas a crear un mejor clasificador de texto para el conjunto de datos de IMDb?**

Como el modelo del lenguaje no requiere pasarle datos etiquetados, estos 50.000 datos extras ayudaran a que el proceso de ajuste cuente con mayor informacion de como realizar las agrupaciones. Luego, este modelo de lenguaje contara con nueva informacion de palabras para el modelo de clasificacion. (en resumidas palabras, tendra un mejor ajuste de los grupos y contara con nuevas palabras para interpretar) 

**8. ¿Cuáles son los tres pasos para preparar sus datos para un modelo de lenguaje?**

1. Definir y aplicar un metodo de tokenization. (en la clase se dan a conocer 3)
2. Numeralizar los tokens.
3. Generar una cadena de texto gigante y segmentarla en trozos (mini-batches). Esto se hace con el modulo de `LMDataLoader`.

**9. ¿Qué es la "tokenización"? ¿Por qué lo necesitamos?**

Con esta podemos identificar, dentro de una cadena de texto, los caracteres o palabras contenidos. Esto es necesario, en el sentido que el modelo no entiende palabras, solo entiende numeros, por lo que, al aplicar estas reglas sobre una cadena de texto, luego se podra numerar las palabras y estas numeraciones luego hacerlas pasar al modelo.

**10. Nombra tres enfoques diferentes para la tokenización.**

 - Word-based
 - Subword-based
 - Character-based

**11. ¿Qué es "xxbos"?**

Este es un indicador que utiliza Fastai para señalar el inicio de una oracion. (me parece que 'bos' es por 'begining of sentence')

**12. Enumere cuatro reglas que fastai aplica al texto durante la tokenización.**

```
xxbos: begining of stream
xxup : upper 'word'
xxmaj: the next word begin with capital letter
xxunk: unknown word
©    : special character
xxrep: repited 'times', 'letter'
```

**13. ¿Por qué los caracteres repetidos se reemplazan por una ficha que muestra el número de repeticiones y el carácter que se repite?**

De esta manera, la matriz de embedding puede codificar informacion mas general sobre conceptos que andar haciendo un token por cada caracter repetido.

**14. ¿Qué es la "numerización"?**

Este es el proceso de transformar los tokens a numero.

1. Primero hacemos una lista de todas los niveles (palabras) que se pueden encontrar (diccionario)
2. Replazar cada uno de estos niveles con el indice del diccionario.

**15. ¿Por qué puede haber palabras que se reemplacen con la ficha de "palabra desconocida"?**

La ficha de palabra desconocida es `xxunk` y esta puede surgir de manera natural, es decir que, al buscar en el diccionario, no se encuentra la palabra, o puede surgir de manera intencionada como el metodo que utiliza fastai para generar los batch en el modelo de clasificacion (como el tamaño de los textos difieren, lo que hace fastai es rellenar con estos caracteres)

**16. Con un batch size de 64, la primera fila del tensor que representa el primer batch contiene los primeros 64 tokens para el conjunto de datos. ¿Qué contiene la segunda fila de ese tensor? ¿Qué contiene la primera fila del segundo batch?.**

Ojo, que en la clase aparece que cada batch contienen, por defecto, 72 tokens y no 64.

Luego, respondiendo a la primera pregunta (y suponiendo 64 tokens en lugar de 72 por batch), en la segunda fila del tensor se encuentra el segundo token del batch.

Luego, la primera fila del segundo batch, tambien contiene un token (y creo que este seria 'xxbos')

Para dejarlo mas claro, aqui dejo el mismo ejercicio aplicado a otro ejemplo.

```python
# generamos unos textos de prueba
texto1 = ["Este texto lo voy a ocupar de prueba y la idea es poder ir registrando los batch y su contenido, para asi corroborar que no estamos haciendo tonteras. "]
texto2 = ["Espero, no confundir con esto, pero tambien en esta parte lo que se hace es agrupar los textos y aplicarles el tokenizer (antes de separar en batches). "]
textos = texto1 + texto2

# luego los pasamos a tokens
spacy = WordTokenizer()
tkn = Tokenizer(spacy)
txts = [tkn(i) for i in textos]

# luego generamos el diccionario 
## nota: si quiero que aparezcan todas las palabras debo fijar min_freq=1
num = Numericalize(min_freq=1)
num.setup(txts)

# luego pasamos los tokens a numero
numeros = [num(i) for i in txts]

# luego cargamos un dataloader y vamos respondiendo a las preguntas
dl = LMDataLoader(numeros, bs=3, seq_len=22)

# antes, miremos que lo hicimos bien (notar que el target contiene el token siguiente)
x,y = first(dl)
## batch1
batch1 = [num.vocab[i] for i in x[0]]
print(batch1[-3:])
>>> ['batch', 'y', 'su']
## target1
target1 = [num.vocab[i] for i in y[0]]
print(target1[-3:])
>>> ['y', 'su', 'contenido']


# Ahora si, respondemos a las preguntas
## 1) que contiene la segunda fila del primer batch?
res1 = x[0][1]
print(f'token a numero: {res1}  token a texto: {num.vocab[res1]}')
>>> token a numero: 8  token a texto: xxmaj


## 2) que contiene la primera fila del segundo batch?
res2 = x[1][0]
print(f'token a numero: {res2}  token a texto: {num.vocab[res2]}')
>>> token a numero: 32  token a texto: contenido
```

**17. ¿Por qué necesitamos relleno para la clasificación de texto? ¿Por qué no lo necesitamos para modelar el lenguaje?**

Lo que sucede es que, cuando generamos los batches para el modelo del lenguaje, lo que se hace es agarrar todos los textos, poner todos los tokens en secuencia y luego cortar en trozos (segun alguna cantidad de tokens). Asi es como generamos los batches del modelo de lenguaje y a gracia es que todos estos tienen la misma cantidad de tokens (lo que es necesario, porque la entrada del modelo debe tener una cantidad fija de datos de entrada)

Por el contrario, para el modelo de clasificacion los batches corresponden a los textos de manera individual con lo cual cada uno varia en la cantidad de tokens que contiene. Asi, una forma sencilla de arreglar esto (y tener una cantidad fija de datos de entrada al modelo) es rellerar los batches con menos cantidad de tokens para asi emparejarlos con el resto.

**18. ¿Qué contiene un embedding matrix para NLP? Cual es su forma?**

Un embedding matrix contiene para cada una de sus filas, cada item del diccionario de palabras. Estas se utilizan al inicio del modelo de redes neuronales, las cuales toman como input los indices del vocabulario (similar a tomar vectores de one-hot encodding, pero mas rapido)

**19. ¿Qué es "perplexity"?**

Es una metrica que generalmente se utiliza para modelos del lenguaje y esta corresponde a la exponencial de la funcion de perdida (cross entropy)

**20. ¿Por qué tenemos que pasar el vocabulario del modelo de lenguaje al bloque de datos del clasificador?**

Porque, el modelo de AWD_LSTM, viene con un embedding matrix dedicado al conjunto de datos de wikipedia, por lo cual, si queremos hacer que calzen los parametros entrenados del encoder (obtenido del modelo de lenguaje con las reseñas de IMDb), necesitamos ajustar nuevamente la arquitectura del modelo AWD_LSTM agreagrando el diccionario de palabras (vocabulario).

**21. ¿Qué es "gradual unfreezing"?**

En la ultima etapa de esta clase utilizamos este concepto al ir actualizando los parametros de las ultimas capas de manera gradual, es decir, primero congelamos todos los parametros salvo la ultima capa (esta puede variar sus parametros), luego congelamos todos los parametros salvo las ultimas dos capas y asi sucesivamente.

**22. ¿Por qué es probable que la generación de texto esté siempre por delante de la identificación automática de textos generados por máquina?**

Al menos, en el texto de la clase, se da a entender que el algoritmo discriminador (verificar si el texto es real o no), requiere estar constantemente actualizandose con informacion fidedigna e informacion generada. LO MALO, es que los algoritmos generadores pueden incluir dentro de su estructura una red discriminadora y asi, esta nueva estructura (que incluye una red generadora y una discriminadora) puede estar constantemente actualizandose para vulnerar la deteccion de otras redes discriminadoras (si no me equivoco este tipo de estructuras de redes neuronales se llaman 'Redes Neuronales Generativas Adversarias')

# **NLP Deep Dive: RNNs**

Lo que vamos a hacer es utilizar una red neuronal, pero en este caso no vamos a con un metodo de aprendizaje profundo, sino que en esta ocacion vamos a utilizar un aprendizaje auto-supervisado (En este caso no necesitamos etiquetar los datos).

Asi, en esta clase vamos a utilizar un modelo preentrenado de lenguaje (predice la siguiente palabra) como base para un modelo de clasificacion de peliculas.

Lo que vamos a hacer.

- Primero, vamos a rescatar el modelo de lenguaje (preentrenado)que se contruyo en base a un dataset de wikipedia
- Luego, vamos a rescatar las reseñas de IMDb (25.000 train, 25.000 validation y 50.000 sin cclasificar) y vamos a hacer un fine-tune con estos datos
- Este metodo es llamado 'Universal Language Model Fine-tuning (ULMFit)' y lo que nos va a generar es un modelo con mayor acierto en predecir la siguiente palabra en una reseña de pelicula.
- Ya, a partir de aqui, generamos el clasificador (tambien con un fine-tuning, pero al modelo de lenguaje modificado)

## **Text Preprocessing**

1. Hacer una lista de todas las palabras (vocabulario)
2. Reemplazar cada palabra, con le indice guardado en el diccionario.
3. Crear un embedding matrix que tenga una fila por cada elemento del diccionario.
4. Usar este embedding matrix como la primera capa del modelo.


In [1]:
!pip install -Uqq fastbook
import fastbook
from fastbook import *
from IPython.display import display,HTML
from fastai.text.all import *


     |████████████████████████████████| 720 kB 5.3 MB/s 
     |████████████████████████████████| 186 kB 44.4 MB/s 
     |████████████████████████████████| 1.2 MB 34.7 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 51 kB 300 kB/s 


### **Tokenization**

Tokenization mas comunes.
 - word-based: separar una oracion por espacios.
 - subword based: separa las oraciones en subgrupos de letras (las que mas se repitan)
 - character-based: separar por letras.

### **Word Tokenization with fastai**

In [ ]:
# Primero descarguemos los datos de las reseñas de IMDb
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
# Ahora quiero corroborar la cantidad de datos
## train
print(len((path/'train/pos').ls()))
print(len((path/'train/neg').ls()))

## no-supervisado (sin etiquetar)
print(len((path/'unsup').ls()))

## test (validacion)
print(len((path/'test/pos').ls()))
print(len((path/'test/neg').ls()))

# Todos estos son los archivos de texto (reseñas) que tengo que juntar

12500
12500
50000
12500
12500


In [ ]:
# Esto lo podemos hacer rapidamente con el metodo get_text_files, el cual buscar
# todos los archivos de texto contedinos del las carpetas seleccionadas
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [ ]:
files

(#100000) [Path('/root/.fastai/data/imdb/test/pos/2456_10.txt'),Path('/root/.fastai/data/imdb/test/pos/9325_9.txt'),Path('/root/.fastai/data/imdb/test/pos/1384_9.txt'),Path('/root/.fastai/data/imdb/test/pos/8457_9.txt'),Path('/root/.fastai/data/imdb/test/pos/9908_7.txt'),Path('/root/.fastai/data/imdb/test/pos/9165_8.txt'),Path('/root/.fastai/data/imdb/test/pos/4981_8.txt'),Path('/root/.fastai/data/imdb/test/pos/6976_10.txt'),Path('/root/.fastai/data/imdb/test/pos/11426_7.txt'),Path('/root/.fastai/data/imdb/test/pos/3618_10.txt')...]

In [ ]:
# Aqui, por ejemplo, podemos ver la primera reseña 
txt = files[0].open().read()
txt[:75]

'Estevez, Martin Sheen and Kathy Bates are superb in this portrayal of a Vie'

In [ ]:
# Lo que vamos a hacer aqui es segmentar las oraciones por espacios (con algunas reglas extras)
spacy = WordTokenizer() # separador por espacios


toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#165) ['Estevez',',','Martin','Sheen','and','Kathy','Bates','are','superb','in','this','portrayal','of','a','Vietnam','vet','home','from','the','war','but','still','haunted','by','it','.','Bates','plays','a','clueless'...]


In [ ]:
# Probemos con otro ejemplo
test = '''Probemos con esta oracion de prueba, PD: es interesante ver como trabaja
para el español'''
print(test)

a = first(spacy([test]))
coll_repr(a, 30)

Probemos con esta oracion de prueba, PD: es interesante ver como trabaja
para el español


"(#18) ['Probemos','con','esta','oracion','de','prueba',',','PD',':','es','interesante','ver','como','trabaja','\\n','para','el','español']"

In [ ]:
# Bien, ahora vamos a aplicar un metodo que proportciona fastai para identificar
# las operaciones que realiza sobre las oraciones
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))
print(coll_repr(tkn(test), 31))

(#184) ['xxbos','xxmaj','estevez',',','xxmaj','martin','xxmaj','sheen','and','xxmaj','kathy','xxmaj','bates','are','superb','in','this','portrayal','of','a','xxmaj','vietnam','vet','home','from','the','war','but','still','haunted','by'...]
(#21) ['xxbos','xxmaj','probemos','con','esta','oracion','de','prueba',',','xxup','pd',':','es','interesante','ver','como','trabaja','\n','para','el','español']


In [ ]:
prueba = L(test[:20]).map(tkn)
prueba

(#1) [['xxbos', 'xxmaj', 'probemos', 'con', 'esta', 'or']]

In [ ]:
# estas son las reglas adicionales que se aplican
defaults.text_proc_rules

[<function fastai.text.core.fix_html>,
 <function fastai.text.core.replace_rep>,
 <function fastai.text.core.replace_wrep>,
 <function fastai.text.core.spec_add_spaces>,
 <function fastai.text.core.rm_useless_spaces>,
 <function fastai.text.core.replace_all_caps>,
 <function fastai.text.core.replace_maj>,
 <function fastai.text.core.lowercase>]

```
Algunas de las notaciones:
xxbos: begining of sentence
xxbaj: 
xxup : upper 'word'
©    : special character
xxrep: repited 'times', 'letter'
```

In [ ]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

### **Subword Tokenization**

1. Analizar el cuerpo del texto y hallar los gropos mas fercuentes de letras (estos luego pasan a funcionar como el diccionario)

2. Aplicar una separacion del texto, mirando el vocabulario creado.

In [ ]:
# Por ejemplo, vamos a utilizar este texto
txts = L(o.open().read() for o in files[:2000])

In [ ]:
# Lo que vamos a hacer aqui es denifir el tamaño del diccionario con el cual
# queremos trabajar (vocab_sz), para luego generar las segmentaciones
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [ ]:
# Por ejemplo, si quisieramos trabajar con un diccionario de tamaño 1000
subword(1000)

"▁Qu it e ▁simply , ▁one ▁of ▁the ▁sh ar p est ▁screen play s ▁ever ▁written ▁and ▁ lu ck i ly ▁it ▁has ▁performance s ▁to ▁m atch . ▁This ▁movie , ▁and ▁if ▁you ▁have n '"

In [ ]:
# O uno de tamaño 200 (aqui se nota que le cuesta mas hacer grupos de letras mas grandes)
subword(200)

'▁ Q u it e ▁s i m p ly , ▁on e ▁of ▁the ▁s h ar p es t ▁s c re en p la y s ▁ e ver ▁w ri t t en ▁and ▁ l'

In [ ]:
# Por el contrario, si aumentamos el tamaño del diccionario, se podran generar
# grupos de letras mas grandes (alcanzando a cubrir palabras)
subword(10000)

"▁Qui te ▁simply , ▁one ▁of ▁the ▁sharp est ▁screenplay s ▁ever ▁written ▁and ▁luck ily ▁it ▁has ▁performances ▁to ▁match . ▁This ▁movie , ▁and ▁if ▁you ▁haven ' t ▁seen ▁it ▁then ▁you ▁really ▁should ▁rush ▁out ▁to"

### **Numericalization with fastai**

1. Hacer una lista con todas las palabras enumeradas (diccionario)
2. Reemplazar cada palabra con su numeracion

In [ ]:
# Veamos el siguiente ejemplo
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#184) ['xxbos','xxmaj','estevez',',','xxmaj','martin','xxmaj','sheen','and','xxmaj','kathy','xxmaj','bates','are','superb','in','this','portrayal','of','a','xxmaj','vietnam','vet','home','from','the','war','but','still','haunted','by'...]


In [ ]:
# Con la funcion map, puedo aplicar una funcion mirando los elementos
toks200 = txts[:200].map(tkn)
toks200[0]

(#184) ['xxbos','xxmaj','estevez',',','xxmaj','martin','xxmaj','sheen','and','xxmaj'...]

In [ ]:
# Creamos el objeto Numericalize (es de Fastai)
# Luego, a este objeto le cargamos el texto y listo
## Ojo, que el objeto Numericalize tiene por defecto un diccionario maximo de 60.000
## En este caso, si dejamos este parametro tal como esta y resulta que en realidad
## existen mas de 60.000 palabras en el diccionario, todas las extras se cambiaran por
## 'xxunk', ademas existe otro parametro min_frec=3, que tambien reemplaza las palabras
## por 'xxunk', pero las que aparezcan menos de 3 veces en el texto
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1992) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','it','in','i'...]"

In [ ]:
# Aqui, por ejemplo podemos ver como dejo la anterior frase commo una secuencia 
# de numeros
nums = num(toks)[:20]; nums

TensorText([   2,    8,  827,   10,    8,  718,    8,  719,   12,    8, 1487,    8,  965,   42,  630,   18,   20,  828,   14,   13])

In [ ]:
# Ahora, si queremos volver a armar la frase
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj quite simply , one of the xxunk xxunk ever written and xxunk it has performances to match .'

### **Putting Our Texts into Batches for a Language Model**

La principal dificultad que se presenta ahora es que, no todos los textos contienen la misma cantidad de palabras y necesitamos una cantidad estructurada de informacion que entre al modelo.

In [ ]:
# Por ejemplo, en este texto, podriamos tratar de generar 6 muestras (batches)
# con 15 palabras cada una
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [ ]:
# Lo que vamos a hacer ahora es dividir lo que tenemos arriba en 3 grupos de 5 palabras
## GRUPO 1 (Primeras 5 columnas)
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [ ]:
## GRUPO 2 (Desde columna 5 a columna 9)
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [ ]:
## GRUPO 3 (Desde columna 10 a 14)
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [ ]:
# Con este paso, vamos a numeralizar todos los textos
nums200 = toks200.map(num)

In [ ]:
# Luego lo pasamos a un datalooader
# Este dataloader primero desordena los documentos (no las oraciones)y luego las
# segmenta segun el tamaño del batch-size que especifiquemos
dl = LMDataLoader(nums200)

In [ ]:
# Aqui por ejemplo podemos ver que se esta utilzando por defecto un BS de 64
# y 72 palabras para cada muestra
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [ ]:
# Por ejemplo, aqui podemos ver el primer input
x[0]

LMTensorText([   2,    8,  827,   10,    8,  718,    8,  719,   12,    8, 1487,    8,  965,   42,  630,   18,   20,  828,   14,   13,    8,  966, 1488,  259,   51,    9,  144,   31,  103, 1489,   43,   17,
          11,    8,  965,  223,   13,    0,  294,   49,   64,  501,    9,  129,   15,   44,   13,   22,  129,   11,   22,    8,  719,   16, 1175,   24,    9,  295,   49,  375,   15,  558,   56,   34,
         344,   16,  181,  195,   31,   16,  115,    0])

In [ ]:
# Esto es lo que dice, si lo volvemos a codificar en palabras
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj quite simply , one of the xxunk xxunk ever written and xxunk it has performances to match .'

In [ ]:
# Haciendo lo mismo para el target (y), podemos ver que la unica diferencia es 
# que esta incluye la ultima palabra que faltaria predecir de la frase
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj quite simply , one of the xxunk xxunk ever written and xxunk it has performances to match . xxmaj'

## **Training a Text Classifier**

### **Language Model Using DataBlock**

In [ ]:
# Aqui, cargo los textos
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

# Ojo que al ejecutar el entrenamiento me salio el mensaje de 'out of memory'
# Asi que realice los siguientes cambios
# tuve que bajar el bs (128 -> 16)
# y el seq_len (80 -> 40)
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=16, seq_len=40) 

In [ ]:
# Al igual que el resto de los dataloaders, podemos ver un batch 
# con el metodo show_batch
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos i xxmaj like the horror genre and xxup the xxup girl xxup next xxup door is on a different level . \n\n xxmaj this movie is so realistic to the teeth , that it makes you think about life","i xxmaj like the horror genre and xxup the xxup girl xxup next xxup door is on a different level . \n\n xxmaj this movie is so realistic to the teeth , that it makes you think about life ."
1,never have been acceptable to any audience . xxmaj now that does n't mean all viewers will appreciate this film . xxmaj no this little hummer is not for the weak at heart or the homophobic . xxmaj in the,have been acceptable to any audience . xxmaj now that does n't mean all viewers will appreciate this film . xxmaj no this little hummer is not for the weak at heart or the homophobic . xxmaj in the end


### **Fine-Tuning the Language Model**

In [ ]:
# Aqui comenzamos el entranamiento
# Esto que estamos ajustando es similar a un modelo de clasificacion, ya que la
# palabra que vamos a predecir se debe encontrar en el diccionario que generamos
# antes 


## Notemos que la estructura de red neuronal que vamos a ocupar es AWD_LSTM
## Esta posee una primera capa de embedding y otras dos capas densas
## Tambien es importante mencionar que utiliza otro optimizador (Adam)
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
# Notar que a pesar de haber reducido el batch size y haber aplicado el metodo
# to_fp16() el tiempo de entrenamiento fue bastante extenso

# Recordar que con el metodo de fit_one_cycle, cambiamos la capa de salida (en 
# este con nuestro diccionario) y congelamos todos los parametros salvo los 
# de la ultima capa (solo se modifican los parametros de la capa de salida)
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,8.240455,7.952467,0.058182,2842.578857,2:58:23


### **Saving and Loading Models**

In [ ]:
# Dado que me tomo tanto tiempo hacer solo una pasada esta la voy a guardar y 
# utilizar mas tarde (este archivo peso mas 300 mb, con lo cual seria bueno tenerlo
# en el drive si se quiere ocupar despues)
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
learn.save('/content/1epoch')

Path('/content/1epoch.pth')

In [ ]:
# Volvemos a cargar el modelo guardado
learn = learn.load('1epoch')

In [ ]:
# Aqui podriamos continuar ajustando el modelo, pero en mi caso me tomo cerca de 3 
# horas hacer una pasada, asi que por el momento esto lo dejare pendiente 
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: ignored

In [ ]:
# Por ultimo, vamos a guardar todo el modelo salvo la ultima capa (recordardo que
# esta transformaria la salida en tipos de probabilidad por la softmax)
## El modelo sin la ultima capa, pasa a llamarse 'encoder'
learn.save_encoder('finetuned')

### **Text Generation**

In [ ]:
# Antes de continuar, miremos como el modelo anterior logra hacer predicciones
# para cantinuar la siguiente oracion
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

i liked this movie because , and how The better is , and as was the film is , as from Starkey in the brilliant And Lowdown ( what is , the Coz is at the new the more the
i liked this movie because of this just . a , , drinking that Bars really i , the game it , with The likely and farmhouse was was of a of over , after One football 's 's -


### **Creating the Classifier DataLoaders**

Lo que sigue ahora es utilizar el modelo de lenguaje preentrenado con los datos de las reseñas y adaptarlo a uno de clasificacion de sentimientos (en este caso, positivo y negativo)

In [ ]:
# Asi es como nosotros, en alguna clase anterior, cargamos el modelo de las reseñas
## Ojo que ya no estamos utilizando los datos no etiquetados (unsupervised)
## Tambien notar que en 'TextBlock.from_folder' estamos utilizando el diccionario
## nuevo (is_lm=False porque no vamos a trabajar con un lenguage model)
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab, is_lm=False),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
# Miremos como vienen armadas estas muestras (input, target)
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxmaj by now you 've probably heard a bit about the new xxmaj disney dub of xxmaj miyazaki 's classic film , xxmaj laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky . xxmaj during late summer of 1998 , xxmaj disney released "" kiki 's xxmaj delivery xxmaj service "" on video which included a preview of the xxmaj laputa dub saying it was due out in "" 1 xxrep 3 9 "" . xxmaj it 's obviously way past that year now , but the dub has been finally completed . xxmaj and it 's not "" laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky "" , just "" castle xxmaj in xxmaj the xxmaj sky "" for the dub , since xxmaj laputa is not such a nice word in xxmaj spanish ( even though they use the word xxmaj laputa many times",pos


In [ ]:
# Otra cosa que es importante de mencionar, es que la longitud de los textos varian
# y en este caso, no podemos sumar todos los textos y luego seccionarlos.
# Lo que se hace es rellenar los textos con menos palabras y aparte los mini-batch
# los intenta generar de manera es estos tengan textos de tamaño similar

## Ahora volvemos a generar un modelo AWD_LSTM nuevo
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [ ]:
# Sin embargo, le vamos a cargar esto (para ajustar casi todos los pametros)
learn = learn.load_encoder('finetuned')

### **Fine-Tuning the Classifier**

Para los problemas de clasificacion de imagenes, lo que se hacia era congelar las primeras capas, entrenar la ultima, luego descongelar todo y entrenar normalmente.

En este caso, se han hayado mejores resultados trabajando de la manera que sigue.

In [ ]:
# Trabajamos la ultima capa
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.682537,0.672927,0.585200,08:06


In [ ]:
# Luego trabajamos las ultimas dos capas, con intervalos de LR
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.645628,0.638529,0.638760,09:18


In [ ]:
# Luego trabajamos las ultimas 3 capas, tambien con intervalos de LR
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.612152,0.576697,0.705600,13:53


In [ ]:
# Por ultimo, descongelamos todo y hacemos el entrenamiento de todas las capas
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
